## Olympics Prediction Model

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [3]:
!pip install tensorflow[and-cuda]

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 234.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 107.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 251.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 245.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 245.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 254.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 245.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 324.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 260.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 317.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 332.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 194.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 243.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 213.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 MB 210.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [1]:
# Import packages

from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Matplotlib created a temporary cache directory at /tmp/matplotlib-1qibefqn because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix

In [38]:
#Import Data

snowflake.get_connection(connection_name="ME_OLYMPICS_CNX")
data = get_dataframe("ATHLETE_EVENTS")
data

Exception occurred in getting snowflake connection: 'connectionSources'
Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


In [42]:
data['AGE'].value_counts().to_dict()

{'23': 21875,
 '24': 21720,
 '22': 20814,
 '25': 19707,
 '21': 19164,
 '26': 17675,
 '27': 16025,
 '20': 15258,
 '28': 14043,
 '19': 11643,
 '29': 11463,
 '30': 9488,
 'NA': 9474,
 '18': 8152,
 '31': 7559,
 '32': 6246,
 '17': 5376,
 '33': 4800,
 '34': 3985,
 '16': 3852,
 '35': 3133,
 '36': 2503,
 '15': 2203,
 '37': 1953,
 '38': 1612,
 '39': 1405,
 '40': 1210,
 '41': 953,
 '42': 866,
 '14': 837,
 '43': 793,
 '44': 683,
 '45': 584,
 '46': 429,
 '47': 408,
 '48': 407,
 '49': 362,
 '50': 278,
 '52': 244,
 '53': 200,
 '51': 199,
 '13': 187,
 '54': 162,
 '56': 131,
 '55': 104,
 '60': 88,
 '59': 87,
 '58': 84,
 '65': 84,
 '57': 69,
 '61': 68,
 '62': 62,
 '69': 60,
 '63': 56,
 '12': 39,
 '71': 33,
 '66': 31,
 '64': 30,
 '70': 28,
 '67': 25,
 '68': 25,
 '72': 24,
 '11': 13,
 '74': 12,
 '73': 8,
 '76': 7,
 '75': 4,
 '88': 3,
 '80': 3,
 '81': 2,
 '77': 2,
 '84': 1,
 '96': 1,
 '10': 1,
 '97': 1}

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      271116 non-null  int32 
 1   NAME    271116 non-null  object
 2   SEX     271116 non-null  object
 3   AGE     271116 non-null  object
 4   HEIGHT  271116 non-null  object
 5   WEIGHT  271116 non-null  object
 6   TEAM    271116 non-null  object
 7   NOC     271116 non-null  object
 8   GAMES   271116 non-null  object
 9   YEAR    271116 non-null  int16 
 10  SEASON  271116 non-null  object
 11  CITY    271116 non-null  object
 12  SPORT   271116 non-null  object
 13  EVENT   271116 non-null  object
 14  MEDAL   271116 non-null  object
dtypes: int16(1), int32(1), object(13)
memory usage: 28.4+ MB


In [47]:
data['AGE'][data['AGE']!='NA'].astype(int).median()

24.0

In [51]:
data['AGE'].replace('NA',int(data['AGE'][data['AGE']!='NA'].astype(int).median()))

0         24
1         23
2         24
3         34
4         21
          ..
271111    29
271112    27
271113    27
271114    30
271115    34
Name: AGE, Length: 271116, dtype: object

In [57]:
data['AGE'] = data['AGE'].replace('NA',str(int(data['AGE'][data['AGE']!='NA'].astype(int).median()))).astype(int)

In [59]:
data['HEIGHT'] = data['HEIGHT'].replace('NA',str(int(data['HEIGHT'][data['HEIGHT']!='NA'].astype(int).median()))).astype(int)

In [63]:
data['WEIGHT'] = data['WEIGHT'].replace('NA',76).astype(int)

ValueError: invalid literal for int() with base 10: '76.5'

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      271116 non-null  int32 
 1   NAME    271116 non-null  object
 2   SEX     271116 non-null  object
 3   AGE     271116 non-null  int64 
 4   HEIGHT  271116 non-null  object
 5   WEIGHT  271116 non-null  object
 6   TEAM    271116 non-null  object
 7   NOC     271116 non-null  object
 8   GAMES   271116 non-null  object
 9   YEAR    271116 non-null  int16 
 10  SEASON  271116 non-null  object
 11  CITY    271116 non-null  object
 12  SPORT   271116 non-null  object
 13  EVENT   271116 non-null  object
 14  MEDAL   271116 non-null  object
dtypes: int16(1), int32(1), int64(1), object(12)
memory usage: 28.4+ MB


In [39]:
data_bkp = get_dataframe("ATHLETE_EVENTS")
data_bkp

Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


In [6]:
tf.random.set_seed(100)

In [34]:
data= data_bkp

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      271116 non-null  int32 
 1   name    271116 non-null  object
 2   sex     271116 non-null  object
 3   age     271116 non-null  int64 
 4   height  271116 non-null  object
 5   weight  271116 non-null  object
 6   team    271116 non-null  object
 7   noc     271116 non-null  object
 8   games   271116 non-null  object
 9   year    271116 non-null  int16 
 10  season  271116 non-null  object
 11  city    271116 non-null  object
 12  sport   271116 non-null  object
 13  event   271116 non-null  object
 14  medal   271116 non-null  int64 
dtypes: int16(1), int32(1), int64(2), object(11)
memory usage: 28.4+ MB


In [36]:
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      271116 non-null  int32 
 1   name    271116 non-null  object
 2   sex     271116 non-null  object
 3   age     271116 non-null  int64 
 4   height  271116 non-null  object
 5   weight  271116 non-null  object
 6   team    271116 non-null  object
 7   noc     271116 non-null  object
 8   games   271116 non-null  object
 9   year    271116 non-null  int16 
 10  season  271116 non-null  object
 11  city    271116 non-null  object
 12  sport   271116 non-null  object
 13  event   271116 non-null  object
 14  medal   271116 non-null  int64 
dtypes: int16(1), int32(1), int64(2), object(11)
memory usage: 28.4+ MB


# CREATING LABELS

In [30]:
data['medal'] = data['medal'].apply(lambda x: 1 if str(x) != 'nan' else 0)

In [37]:
data['age'].value_counts()

age
1    271116
Name: count, dtype: int64

In [31]:
data['age'] = data['age'].apply(lambda x: 1 if str(x) != 'NA' else 0)
data['age']

0         1
1         1
2         1
3         1
4         1
         ..
271111    1
271112    1
271113    1
271114    1
271115    1
Name: age, Length: 271116, dtype: int64

In [29]:
data['age'].max()

1

In [16]:
data['height'] = data['height'].apply(lambda x: 1 if str(x) != 'NA' else data.median())
data['height'] = data['height'].apply(lambda x: 1 if str(x) != 'NA' else data.median())

TypeError: could not convert string to float: 'M'

# Cleaning data

In [15]:
# converting datatypes
data = data.astype({'age': int, 'height': int, 'weight': int})
print(data.dtypes)


ValueError: invalid literal for int() with base 10: 'NA': Error while type casting for column 'height'

In [10]:
data = data.drop(['id', 'name', 'games'], axis=1)

In [11]:
data.isna().mean()

sex       0.0
age       0.0
height    0.0
weight    0.0
team      0.0
noc       0.0
year      0.0
season    0.0
city      0.0
sport     0.0
event     0.0
medal     0.0
dtype: float64

In [12]:
data.groupby(['medal', 'sex']).mean().astype(np.int)

TypeError: Could not convert ... to numeric

In [ ]:
for column in ['Age', 'Height', 'Weight']:
    data[column] = data.groupby(['Medal', 'Sex'])[column].apply(lambda x: x.fillna(x.mean()).astype(np.int))